In [ ]:
!pip install ultralytics


In [ ]:
!pip install tracker

In [ ]:
!pip install supervision

In [ ]:
#Tạm thời đang lỗi
!pip install pytube
!mkdir img_croped
from pytube import YouTube
def download_video_from_youtube(link, path):
    yt = YouTube(link)
    video = yt.streams.get_highest_resolution()
    video.download(path)
download_video_from_youtube('https://www.youtube.com/watch?v=WvhYuDvH17I', '/content/img_croped')

In [ ]:
# Import
from ultralytics import YOLO
import cv2
import numpy as np
import os
from PIL import Image
import supervision as sv
# from tracker import Tracker


In [ ]:
#!yolo detect train data=coco128.yaml model=yolov8n.pt epochs=100 imgsz=640 device=mps
model = YOLO('yolov8x.pt')
results = model.predict('https://ultralytics.com/images/bus.jpg', classes = [0])

100%|██████████| 131M/131M [00:03<00:00, 38.1MB/s]

100%|██████████| 476k/476k [00:00<00:00, 3.17MB/s]
image 1/1 /content/bus.jpg: 640x480 5 persons, 4160.2ms
Speed: 20.4ms preprocess, 4160.2ms inference, 34.1ms postprocess per image at shape (1, 3, 640, 480)


In [ ]:
!rm *.jpg

In [ ]:
def draw_in_img():
  for r in results:
      im_array = r.plot()  # plot a BGR numpy array of predictions
      im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
      #im.show()  # show image
      im.save('results.jpg')  # save image
draw_in_img()

In [ ]:
results[0].names[0]

'person'

In [ ]:
results[0].boxes.cls.numpy()

array([          5,           0,           0,           0,           0,           0,           1], dtype=float32)

In [ ]:
results[0].boxes.xyxy.int().numpy()

array([[   1,  230,  804,  741],
       [ 668,  392,  809,  879],
       [  50,  398,  247,  900],
       [ 221,  406,  343,  860],
       [   0,  550,   78,  872],
       [   0,  550,   79, 1064],
       [ 666,   15,  748,   90]], dtype=int32)

In [ ]:

def detect_img(results):
  i = 0
  img = cv2.imread(source)
  bboxs = results[0].boxes
  for j, bbox in enumerate(bboxs):
    if bbox.cls == 0:
      bb = bbox.xyxy[0].int().numpy()
      im = img[bb[1]:bb[3], bb[0]:bb[2]]
      cv2.imwrite(f'{i}_{j}.jpg', im)

detect_img(results)


Video


In [ ]:
result = source = '.mp4'
results = model(source)


In [ ]:
!mkdir img_croped

mkdir: cannot create directory ‘img_croped’: File exists


In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

In [ ]:
!mkdir img_croped
cap = cv2.VideoCapture('/content/y2meta.com-YAMAHA GREEN SLOW MOBILITY で観光地をもっと自由に　もっと楽しく　Short ver.-(1080p).mp4')

if (cap.isOpened()== False):
	print("Error opening video file")

i = 0
while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    print(frame)
    cv2.imwrite(f'content/frame/{i}.jpg', frame)
    result = model(frame)[0]
    for j, bbox in enumerate(result.boxes.xyxy.int().numpy()):
      print(bbox)
      print(4)
      img = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
      cv2.imwrite(f'img_croped/{i}_{j}.jpg', img)
    print(result.boxes.xyxy.int().numpy())
    #cv2.imshow('Frame', frame)
    # if cv2.waitKey(25) & 0xFF == ord('q'):
  #       break
  # else:
  #   break
  i += 1

  if i > 100:
    break


cap.release()

#cv2.destroyAllWindows()

In [ ]:
model = YOLO('yolov8n.pt')
source = '/content/y2meta.com-YAMAHA GREEN SLOW MOBILITY で観光地をもっと自由に　もっと楽しく　Short ver.-(1080p).mp4'
results = model(source, stream=True)

100%|██████████| 6.23M/6.23M [00:00<00:00, 76.5MB/s]


In [ ]:
path = os.path.join('.', 'data', '/content/y2meta.com-YAMAHA GREEN SLOW MOBILITY で観光地をもっと自由に　もっと楽しく　Short ver.-(1080p).mp4')

cap = cv2.VideoCapture(path)
ret, frame = cap.read()

model = YOLO('yolov8n.pt')
while ret:
  results = model(frame)

  for result in results:
    detecttion = []
    for r in result.boxes.data.tolist().int():
      x1, y1, x2, y2, score, class_id = r
      detecttion.append([x1, y1, x2, y2, score])

  #cv2.imshow('frame', frame)
  cv2.waitKey(27)


In [ ]:
from ultralytics import YOLO
import cv2
from ultralytics.utils.plotting import Annotator

In [ ]:
path = "/content/img_croped/Shopping People Commerce Mall Many Crowd Walking   Free Stock video footage   YouTube.mp4"

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

In [ ]:
model = YOLO('yolov8m.pt')
cap = cv2.VideoCapture(path)

while True:
    _, img = cap.read()

    results = model.predict(img, classes =[0], verbose = True)

    for r in results:

        annotator = Annotator(img)

        boxes = r.boxes
        for box in boxes:

            b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
            c = box.cls
            annotator.box_label(b, model.names[int(c)])

    img = annotator.result()
    #cv2.imshow('YOLO V8 Detection', img)
    out.write(img)
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

cap.release()
cv2.destroyAllWindows()

Tracking


In [ ]:
from logging import config
def trackkk():
  model = YOLO('yolov8m.pt')

  box_annotator = sv.BoxAnnotator{
      thickness = 2,
      text_thickness = 1,
      text_scale=0.5
  }
  for result in model.track(source="/content/img_croped/Shopping People Commerce Mall Many Crowd Walking   Free Stock video footage   YouTube.mp4", conf=0.3, iou=0.7, show=True, classes = [0]):

    frame = result.orig_img
    detections = sv.Detections.from_yolov8(result)

    if result.boxes.id is not None:
      datections.tracker_id = result.boxes.id.cpu().numpy().astype(int)

    detections = detections[detections.class_id == 0]
    labels = {
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for _, confidence, class_is, tracker_id
        in detections
    }

    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)

    cv2.imshow('yolov8', frame)


    if (cv.waitKey(30) == 27):
      break